In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
doc_embeddings = retriever.encode(latex_docs)

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

query_embedding = retriever.encode([latex_query])[0]
_, I = index.search(np.array([query_embedding]), k=5)

retrieved_latex = [latex_docs[i] for i in I[0]]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

context = "\n".join([f"[DOC_{i+1}] {doc}" for i, doc in enumerate(retrieved_latex)])
input_text = f"{context}\n\n[QUERY] {latex_query}"
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

outputs = model.generate(**inputs, max_new_tokens=512)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)